# Build Environment Security Use Case

Automate security monitoring and guidance during the software development lifecycle (SDLC) by embedding a GitHub Action that triggers on pull requests (PRs). The system summarizes changes, evaluates them for security risks, and provides actionable recommendations to reviewers. This turns every PR into an opportunity for proactive security posture improvement — not just static scanning, but contextual reasoning.

## Model used for this use case
Both Instruct Model and Reasoning Model would be suitable for this task. In this example, we used the Instruct Model via SageMaker endpoint.

**Note**: Update the configuration variables below to match your deployment.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# Update these variables to match your deployment
endpoint_name = 'foundation-sec-8b-endpoint'
aws_region = 'us-east-1'

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

## Setup

The setup uses SageMaker endpoint instead of loading the model locally.

In [ ]:
import boto3
import json
import re
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments for reproducible outputs
generation_args = {
    "max_new_tokens": 1024,
    "temperature": None,  # None means deterministic (temperature=0)
    "repetition_penalty": 1.2,
    "do_sample": False,   # Deterministic sampling
    "use_cache": True,
}

print("Generation configuration:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def inference(prompt, system_prompt):
    """Inference function using SageMaker endpoint"""
    
    # Format the conversation for the model
    formatted_prompt = f"System: {system_prompt}\n\nUser: {prompt}\n\nAssistant: "
    
    # Prepare payload for SageMaker endpoint
    payload = {
        "inputs": formatted_prompt,
        "parameters": generation_args
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different TGI response formats
        if isinstance(result, list) and len(result) > 0:
            generated_text = result[0].get('generated_text', '')
        elif isinstance(result, dict):
            generated_text = result.get('generated_text', str(result))
        else:
            generated_text = str(result)
        
        # Clean up the response (remove the original prompt if it's included)
        if generated_text.startswith(formatted_prompt):
            response_text = generated_text[len(formatted_prompt):].strip()
        else:
            response_text = generated_text.strip()
            
        # Remove any trailing special tokens
        response_text = re.sub(r'<\|.*?\|>$', '', response_text).strip()
        
        return response_text
        
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the inference function
test_response = inference("Hello, can you help with security code review?", "You are a security expert.")
print("Test Response:")
print(test_response)

## Security Analysis Functions

In [ ]:
SYSTEM_PROMPT = "You are a security expert reviewing changes introduced in a pull request."

def make_prompt(pr_diff_text):
    return f'''Analyze the following code diff and identify:
    1. Security-relevant changes
    2. Any potential vulnerabilities introduced
    3. A clear summary of affected areas
    4. Recommended remediations (if needed)
    
    ## PULL REQUEST DIFF
    {pr_diff_text}'''

print("Security analysis functions ready!")

## Example: Security-Positive Pull Request Analysis

In this sample PR Diff, we replaced use of `eval()` with `ast.literal_eval()` — this is a significant security improvement. `eval()` can execute arbitrary code, whereas `ast.literal_eval()` safely evaluates strings with Python literals.

In [ ]:
pr_diff_text = """
diff --git a/app/utils.py b/app/utils.py
index e69de29..b25a3c0 100644
--- a/app/utils.py
+++ b/app/utils.py
@@ def process_input(data):
-    return eval(data)
+    import ast
+    return ast.literal_eval(data)

diff --git a/.github/workflows/deploy.yml b/.github/workflows/deploy.yml
index a1b2c3d..d4e5f6g 100644
--- a/.github/workflows/deploy.yml
+++ b/.github/workflows/deploy.yml
@@ steps:
-      run: npm run deploy
+      run: |
+        set -e
+        npm run lint
+        npm test
+        npm run deploy
"""

print("Sample PR diff loaded - this shows a security improvement!")

In [ ]:
print("=== SECURITY ANALYSIS OF PULL REQUEST ===")
response = inference(make_prompt(pr_diff_text), SYSTEM_PROMPT)
display(Markdown(response))

## Example: Analyzing Risky Changes

Let's analyze a different PR that introduces potential security risks:

In [ ]:
risky_pr_diff = """
diff --git a/app/auth.py b/app/auth.py
index abc123..def456 100644
--- a/app/auth.py
+++ b/app/auth.py
@@ def login(username, password):
-    query = "SELECT * FROM users WHERE username = %s AND password = %s"
-    return db.execute(query, (username, password))
+    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
+    return db.execute(query)

diff --git a/app/config.py b/app/config.py
index 111222..333444 100644
--- a/app/config.py
+++ b/app/config.py
@@ class Config:
+    DEBUG = True
+    SECRET_KEY = "hardcoded-secret-key-123"
     DATABASE_URL = os.getenv('DATABASE_URL')
"""

print("=== ANALYZING RISKY PULL REQUEST ===")
risky_response = inference(make_prompt(risky_pr_diff), SYSTEM_PROMPT)
display(Markdown(risky_response))

## Custom Security Analysis

Try analyzing your own code diff:

In [ ]:
# Add your own PR diff here for analysis
custom_pr_diff = """
# Paste your git diff output here
# Example:
# diff --git a/your_file.py b/your_file.py
# index abc123..def456 100644
# --- a/your_file.py
# +++ b/your_file.py
# @@ -1,3 +1,4 @@
# +import requests
#  def fetch_data(url):
# -    return "placeholder"
# +    response = requests.get(url, verify=False)
# +    return response.json()
"""

if custom_pr_diff.strip() and not custom_pr_diff.startswith("# Paste your"):
    print("=== ANALYZING YOUR CUSTOM PR ===")
    custom_response = inference(make_prompt(custom_pr_diff), SYSTEM_PROMPT)
    display(Markdown(custom_response))
else:
    print("Add your PR diff in the cell above to analyze it!")

## Advanced: Multi-Factor Security Analysis

Let's create a more comprehensive analysis that looks at multiple security aspects:

In [ ]:
def comprehensive_security_analysis(pr_diff_text):
    """Perform multi-aspect security analysis"""
    
    analyses = {
        "Vulnerability Assessment": {
            "prompt": f"Focus on identifying potential vulnerabilities in this code diff. Look for injection flaws, authentication bypasses, insecure configurations, and data exposure risks:\n\n{pr_diff_text}",
            "system": "You are a vulnerability assessment specialist."
        },
        "Supply Chain Security": {
            "prompt": f"Analyze this diff for supply chain security risks including new dependencies, build process changes, and third-party integrations:\n\n{pr_diff_text}",
            "system": "You are a supply chain security expert."
        },
        "DevSecOps Impact": {
            "prompt": f"Evaluate how these changes impact the overall DevSecOps pipeline, CI/CD security, and deployment practices:\n\n{pr_diff_text}",
            "system": "You are a DevSecOps security consultant."
        }
    }
    
    results = {}
    for analysis_type, config in analyses.items():
        print(f"\n=== {analysis_type.upper()} ===")
        response = inference(config["prompt"], config["system"])
        results[analysis_type] = response
        display(Markdown(f"### {analysis_type}\n{response}"))
        print("-" * 80)
    
    return results

print("Comprehensive security analysis function ready!")

In [ ]:
# Run comprehensive analysis on the risky PR
print("=== COMPREHENSIVE SECURITY ANALYSIS ===")
comprehensive_results = comprehensive_security_analysis(risky_pr_diff)

## Security Score Generator

Generate a security score and recommendation for the PR:

In [ ]:
def generate_security_score(pr_diff_text):
    """Generate a security score and recommendation for the PR"""
    
    score_prompt = f"""Analyze this pull request diff and provide:
    
    1. A security score from 1-10 (1 = high risk, 10 = excellent security)
    2. A brief risk assessment
    3. A clear recommendation (APPROVE, REQUEST CHANGES, or BLOCK)
    4. Top 3 action items for the developer
    
    Format your response with clear sections and be decisive in your recommendation.
    
    PULL REQUEST DIFF:
    {pr_diff_text}"""
    
    score_system = "You are a senior security engineer providing a final security assessment for code review."
    
    return inference(score_prompt, score_system)

print("=== SECURITY SCORE FOR RISKY PR ===")
security_score = generate_security_score(risky_pr_diff)
display(Markdown(security_score))

## GitHub Integration Example

Here's how you could structure this for GitHub Actions integration:

In [ ]:
def github_pr_security_review(pr_diff, pr_number, repo_name):
    """Complete security review function for GitHub integration"""
    
    print(f"🔍 Security Review for PR #{pr_number} in {repo_name}")
    print("=" * 60)
    
    # Basic security analysis
    basic_analysis = inference(make_prompt(pr_diff), SYSTEM_PROMPT)
    
    # Security score
    score_result = generate_security_score(pr_diff)
    
    # Format for GitHub comment
    github_comment = f"""## 🛡️ Security Review Results

### Analysis Summary
{basic_analysis}

---

### Security Assessment
{score_result}

---
*This review was generated by the Cisco Foundation Security Model*
"""
    
    return {
        "analysis": basic_analysis,
        "score": score_result,
        "github_comment": github_comment
    }

# Example usage
review_result = github_pr_security_review(pr_diff_text, "123", "myorg/myrepo")
print("\n=== GITHUB COMMENT FORMAT ===")
display(Markdown(review_result["github_comment"]))